# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [9]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [10]:
city_data = pd.read_csv('../WeatherPy/city_data.csv')
city_data = city_data.drop(columns=["Unnamed: 0"])
city_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,albany,42.60,-73.97,87.01,35,15,4.41,US,1596225799
1,verkh-usugli,52.70,115.17,59.32,98,100,1.12,RU,1596225857
2,chokurdakh,70.63,147.92,46.89,73,100,2.98,RU,1596225857
3,port elizabeth,-33.92,25.57,51.01,88,0,5.41,ZA,1596225758
4,kapaa,22.08,-159.32,81.00,74,90,17.22,US,1596225857
...,...,...,...,...,...,...,...,...,...
601,fort nelson,58.81,-122.70,78.80,47,20,5.82,CA,1596225838
602,san patricio,19.22,-104.70,87.80,74,40,11.41,MX,1596226033
603,buraydah,26.33,43.98,100.40,24,20,8.05,SA,1596226033
604,pangnirtung,66.15,-65.71,51.80,93,75,12.75,CA,1596225817


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
#Configure gmaps
gmaps.configure(api_key=g_key)

In [12]:
# locations
locations = city_data[["Lat", "Lng"]]

# humidity
humidity = city_data["Humidity"].astype(float)
maxhumidity = humidity.max()


In [19]:
fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating=False, max_intensity=5,
                                   point_radius=1)

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:
#A max temperature lower than 80 degrees but higher than 70 and no Cloudiness
ideal_city_df = city_data.loc[(city_data["Max Temp"] < 80) & (city_data["Max Temp"] > 70) & (city_data["Cloudiness"] == 0), :]
#Drop null values
ideal_city_df = ideal_city_df.dropna(how='any')
#rest indext
ideal_city_df.reset_index(inplace=True)
del ideal_city_df['index']

ideal_city_df.head()
len(ideal_city_df)

23

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [21]:
# Store into variable named hotel_df
hotel_df = pd.DataFrame(ideal_city_df, columns=["City", "Country", "Lat", "Lng"])
#Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,atasu,KZ,48.68,71.64,
1,faanui,PF,-16.48,-151.75,
2,benghazi,LY,32.12,20.07,
3,morondava,MG,-20.28,44.28,
4,fethiye,TR,36.62,29.12,
5,garissa,KE,-0.45,39.64,
6,kutum,SD,14.20,24.67,
7,mlonggo,ID,-6.53,110.70,
8,salta,AR,-24.79,-65.41,
9,watrous,CA,51.67,-105.47,


In [22]:
# Set parameters to search for hotels with 5000 meters.
hotellist = []

for i in range(len(hotel_df)):
    lat = hotel_df.loc[i]['Lat']
    lng = hotel_df.loc[i]['Lng']

    params = {"location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key}
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
   # make request and print url
    name = requests.get(base_url, params=params)
    
    # convert to json
    name = name.json()

    try:
        hotellist.append(name['results'][0]['name'])
    except:
         hotellist.append("")
hotel_df["Hotel Name"] = hotellist
hotel_df = hotel_df.dropna(how='any') 
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,atasu,KZ,48.68,71.64,Atasu
1,faanui,PF,-16.48,-151.75,Vaitape
2,benghazi,LY,32.12,20.07,Benghazi
3,morondava,MG,-20.28,44.28,Morondava
4,fethiye,TR,36.62,29.12,Fethiye
5,garissa,KE,-0.45,39.64,Garissa
6,kutum,SD,14.20,24.67,Kutum
7,mlonggo,ID,-6.53,110.70,Jepara
8,salta,AR,-24.79,-65.41,Salta
9,watrous,CA,51.67,-105.47,Watrous


In [25]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [26]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))